In [1]:
# interpolating for windspeed for all months for all longitudes & latitudes between 0-90dN: named ws_interpolated.nc
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import scipy
from scipy import signal
import netCDF4

ds_ua850 = xr.open_dataset("/rds/general/project/circulates/live/data/20CR/ua850_v2c_185101-201412.nc")
path = '/rds/general/user/ah921/home/UROP_2021/ws_interpolated.nc'
ws_interpolated = netCDF4.Dataset(path,mode='w',format='NETCDF4_CLASSIC')

time = ws_interpolated.createDimension('time', 1968)
lat = ws_interpolated.createDimension('lat', 800)
lon = ws_interpolated.createDimension('lon', 3200) 

times = ws_interpolated.createVariable('time', np.float32, ('time',))
lats = ws_interpolated.createVariable('lat', np.float32, ('lat',))
lons = ws_interpolated.createVariable('lon', np.float32, ('lon',))
value = ws_interpolated.createVariable('value', np.float32, ('time', 'lat', 'lon',))
value.units = 'm/s'
times = ds_ua850.time #potential problemo here 
lons = np.linspace(ds_ua850.lon[0], ds_ua850.lon[127], ds_ua850.dims["lon"]*25)
lats = np.linspace(ds_ua850.lat[0], ds_ua850.lat[31], int(ds_ua850.dims["lat"]*12.5))

for i in range(0,1968):
    ds = ds_ua850.isel(time=i).isel(level=0)
    new_lon = np.linspace(ds.lon[0], ds.lon[127], ds.dims["lon"]*25)
    new_lat = np.linspace(ds.lat[0], ds.lat[31], int(ds.dims["lat"]*12.5)) #bc half of lat
    dsi = ds.interp(lat=new_lat, lon=new_lon)
    yeet = np.dstack(dsi.ua)
    value[i,:,:] = yeet

ws_interpolated.close()
# this bit needs to be here bc otherwise the values come out stupid
ws_interpolated = xr.open_dataset("/rds/general/user/ah921/home/UROP_2021/ws_interpolated.nc")

ws_interpolated['time'] = ds_ua850.time 
ws_interpolated['lon'] = np.linspace(ds_ua850.lon[0], ds_ua850.lon[127], ds_ua850.dims["lon"]*25)
ws_interpolated['lat'] = np.linspace(ds_ua850.lat[0], ds_ua850.lat[31], int(ds_ua850.dims["lat"]*12.5))
print(ws_interpolated.value)


<xarray.DataArray 'value' (time: 1968, lat: 800, lon: 3200)>
[5038080000 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 1851-01-01 1851-02-01 ... 2014-12-01
  * lat      (lat) float64 87.86 87.76 87.65 87.54 ... 1.72 1.612 1.504 1.395
  * lon      (lon) float64 0.0 0.1117 0.2233 0.335 ... 356.9 357.0 357.1 357.2
Attributes:
    units:    m/s


In [ ]:
# interpolating for surface pressure for all months for all longitudes & latitudes between 0-90dN: named ps_interpolated.nc
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import scipy
from scipy import signal
import netCDF4

ds_ps = xr.open_dataset("/rds/general/project/circulates/live/data/20CR/ps_v2c_185101-201412.nc")
path = '/rds/general/user/ah921/home/UROP_2021/ps_interpolated.nc'
ps_interpolated = netCDF4.Dataset(path,mode='w',format='NETCDF4_CLASSIC')

time = ps_interpolated.createDimension('time', 1968)
lat = ps_interpolated.createDimension('lat', 800)
lon = ps_interpolated.createDimension('lon', 3200) 

times = ps_interpolated.createVariable('time', np.float32, ('time',))
lats = ps_interpolated.createVariable('lat', np.float32, ('lat',))
lons = ps_interpolated.createVariable('lon', np.float32, ('lon',))
value = ps_interpolated.createVariable('value', np.float32, ('time', 'lat', 'lon',))
value.units = 'm/s'
times = ds_ps.time #potential problemo here 
lons = np.linspace(ds_ps.lon[0], ds_ps.lon[127], ds_ps.dims["lon"]*25)
lats = np.linspace(ds_ps.lat[0], ds_ps.lat[31], int(ds_ps.dims["lat"]*12.5))
nbndss = ds_ps.nbnds
for i in range(0,1968):
        dsp = ds_ps.isel(time=i)
        new_lon = np.linspace(dsp.lon[0], dsp.lon[127], dsp.dims["lon"]*25)
        new_lat = np.linspace(dsp.lat[0], dsp.lat[31], int(dsp.dims["lat"]*12.5)) #bc half of lat
        dspi = dsp.interp(lat=new_lat, lon=new_lon)
        yeetp = np.dstack(dspi.ps)
        value[i, :,:] = yeetp

ps_interpolated.close()
# this bit needs to be here bc otherwise the values come out stupid
ps_interpolated = xr.open_dataset("/rds/general/user/ah921/home/UROP_2021/ws_interpolated.nc")

ps_interpolated['time'] = ds_ps.time 
ps_interpolated['lon'] = np.linspace(ds_ps.lon[0], ds_ps.lon[127], ds_ps.dims["lon"]*25)
ps_interpolated['lat'] = np.linspace(ds_ps.lat[0], ds_ps.lat[31], int(ds_ps.dims["lat"]*12.5))
print(ps_interpolated.value)


In [14]:
# creating an array for the average windspeed for winter months from interpolated wind speed data, name: average_winds5
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import scipy
from scipy import signal
import netCDF4 as nc
import time

start_time = time.time()

ds_ua850 = xr.open_dataset("/rds/general/project/circulates/live/data/20CR/ua850_v2c_185101-201412.nc")
ws_interpolated = xr.open_dataset("/rds/general/user/ah921/home/UROP_2021/ws_interpolated.nc")

path1 = '/rds/general/user/ah921/home/UROP_2021/average_winds5.nc'
ws_interpolated['time'] = ds_ua850.time 
ws_interpolated['lon'] = np.linspace(ds_ua850.lon[0], ds_ua850.lon[127], ds_ua850.dims["lon"]*25)
ws_interpolated['lat'] = np.linspace(ds_ua850.lat[0], ds_ua850.lat[31], int(ds_ua850.dims["lat"]*12.5))

ua = ws_interpolated.value
# creating the array itself 
average_winds5 = nc.Dataset(path1,mode='w',format='NETCDF4_CLASSIC')

time1 = average_winds5.createDimension('time', 154) 
lat = average_winds5.createDimension('lat', 800) 
lon = average_winds5.createDimension('lon', 3200) 

times = average_winds5.createVariable('time', np.float32, ('time',))
lats = average_winds5.createVariable('lat', np.float32, ('lat',))
lons = average_winds5.createVariable('lon', np.float32, ('lon',))
value = average_winds5.createVariable('value', np.float32, ('time', 'lat', 'lon',))
value.units = 'm/s'
times =  ws_interpolated.time[120:1957:12].dt.year 
lons = np.linspace(ds_ua850.lon[0], ds_ua850.lon[127], ds_ua850.dims["lon"]*25)
lats = np.linspace(ws_interpolated.lat[0], ws_interpolated.lat[31], int(ws_interpolated.dims["lat"]*12.5))

for i in range(10, 164): # years 1861 - 2014 
    for j in range(0,(32*25)): #0dN to 90dN probs 
        for k in range(0,(128*25)): # over all lons 
            value[i-10, j, k] = np.average([ua[(12*i-1):(12*i+2), j, k]])
    print("--- %s seconds ---" % (time.time() - start_time))

            
average_winds5.close()

IndexError: too many indices

In [ ]:
# calculating jet strength, jet latitude & NAO Index
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import scipy
from scipy import signal
import netCDF4 as nc
import time

start_time = time.time()

ds_ua850 = xr.open_dataset("/rds/general/project/circulates/live/data/20CR/ua850_v2c_185101-201412.nc")
ws_interpolated = xr.open_dataset("/rds/general/user/ah921/home/UROP_2021/ws_interpolated.nc")

strength_monthly = np.empty([3])
lat_monthly = np.empty([3])
for i in range(10, 164): # Number of years from 1861 to 2014
    for j in range(0, (32*25)): # Latitude points from 87.9d N to 1.4dN
        for k in range((114*25), (25*128)): # Longitude points from 300.9d E to 357.2d E 
            # Longitude array of mean winter ws at a j-th latitude
            lon_ws_D[k-(114*25)] = ws_interpolated.value[(12*i-1), j, k]
            lon_ws_J[k-(114*25)] = ws_interpolated.value[(12*i), j, k]
            lon_ws_F[k-(114*25)] = ws_interpolated.value[(12*i+1), j, k]
        lat_ws_D[j] = np.average(lon_ws_D) # Latitude array of longitudinal monthly values
        lat_ws_J[j] = np.average(lon_ws_J)
        lat_ws_F[j] = np.average(lon_ws_F)
        lon_ws_D = np.empty([(14*25)])
        lon_ws_F = np.empty([(14*25)])
        lon_ws_J = np.empty([(14*25)])
    jet_strength[i-10] = np.average([max(lat_ws_D), max(lat_ws_J), max(lat_ws_F)])
    jet_latitude[i-10] = np.average([ds_ua850.lat[np.where(lat_ws_D == max(lat_ws_D))], \
                                     ds_ua850.lat[np.where(lat_ws_J == max(lat_ws_J))], \
                                     ds_ua850.lat[np.where(lat_ws_F == max(lat_ws_F))]])
    azores_ps = ds_pressure.ps[(12*i-1):(12*i+2), (8*25), (121*25)] # Azores DJF
    iceland_ps = ds_pressure.ps[(12*i-1):(12*i+2), (18*25), (119*25)] # Iceland DJF
    NAO_index[i-10] = np.average(iceland_ps - azores_ps)/100 # Division by 100 to convert into hPa
    lat_ws_D = np.empty([(32*25)])  
    lat_ws_J = np.empty([(32*25)])  
    lat_ws_F = np.empty([(32*25)])

In [ ]:
# code to get the average ws over time at each point (from the avg winter months windspeed) to then calculate the contours

import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import scipy
from scipy import signal
import netCDF4 as nc
import time

start_time = time.time()

ds_ua850 = xr.open_dataset("/rds/general/project/circulates/live/data/20CR/ua850_v2c_185101-201412.nc")
ws_interpolated = xr.open_dataset("/rds/general/user/ah921/home/UROP_2021/ws_interpolated.nc")
ws_interpolated['time'] = ds_ua850.time 
ws_interpolated['lon'] = np.linspace(ds_ua850.lon[0], ds_ua850.lon[127], ds_ua850.dims["lon"]*25)
ws_interpolated['lat'] = np.linspace(ds_ua850.lat[0], ds_ua850.lat[31], int(ds_ua850.dims["lat"]*12.5))

# creating the array itself 
path2 = '/rds/general/user/ah921/home/UROP_2021/timeavgwinds.nc'
timeavgwinds = nc.Dataset(path2,mode='w',format='NETCDF4_CLASSIC')

lat = timeavgwinds.createDimension('lat', 800) 
lon = timeavgwinds.createDimension('lon', 3200) 
lats = timeavgwinds.createVariable('lat', np.float32, ('lat',))
lons = timeavgwinds.createVariable('lon', np.float32, ('lon',))
value = timeavgwinds.createVariable('value', np.float32, ('lat', 'lon',))
value.units = 'm/s'

lons =  np.linspace(ds_ua850.lon[0], ds_ua850.lon[127], ds_ua850.dims["lon"]*25)
lats = np.linspace(ws_interpolated.lat[0], ws_interpolated.lat[31], int(ws_interpolated.dims["lat"]*12.5))  

for i in range(10, 164): 
    for j in range(0,800): #0dN to 90dN probs 
        for k in range(0,3200): # over all lons
            ds = ws_interpolated.isel(lat=j).isel(lon=k) 
            value[j,k] = np.average(ds.value[(12*i-1):(12*i+2)])
    print("--- %s seconds ---" % (time.time() - start_time))

timeavgwinds.close()
